<a href="https://colab.research.google.com/github/calzateu/Optimization_1/blob/main/Project/Neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
!rm -rf Optimization_1/
!git clone https://github.com/calzateu/Optimization_1.git

Cloning into 'Optimization_1'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 28 (delta 6), reused 14 (delta 3), pack-reused 0
Unpacking objects: 100% (28/28), done.


In [3]:
data = pd.read_excel('Optimization_1/Project/Data_Base.xlsx')

In [4]:
data.head(10)

,C_m_delta_e,U_0,rho,st,c,I_y,C_z_delta_e,m,kp,ki,kd
0,-1.3265,49.408818,1.29,0.252505,0.355711,0.17,-3.565,9242.484970,1.51,2.31,0.06
1,-1.3265,62.106212,1.29,0.322645,0.270942,0.17,-3.565,4661.322645,1.05,1.01,0.01
2,-1.3265,9.134269,1.29,0.591182,0.445892,0.17,-3.565,5977.955912,3.51,5.06,1.01
3,-1.3265,30.561122,1.29,0.897796,0.408016,0.17,-3.565,7979.959920,4.66,16.86,1.01
4,-1.3265,60.320641,1.29,0.557114,0.184369,0.17,-3.565,5923.847695,1.05,1.26,0.01
5,-1.3265,99.206413,1.29,0.482966,0.358417,0.17,-3.565,5328.657315,1.01,3.06,0.01
6,-1.3265,75.002004,1.29,0.210421,0.161824,0.17,-3.565,1865.731463,1.51,1.26,0.06
7,-1.3265,71.232465,1.29,0.855711,0.405311,0.17,-3.565,7691.382766,1.06,3.96,0.01
8,-1.3265,11.316633,1.29,0.521042,0.431463,0.17,-3.565,8593.186373,6.21,4.61,1.01
9,-1.3265,38.298597,1.29,0.272545,0.105912,0.17,-3.565,7222.444890,5.56,7.86,1.01
